In [1]:
from pydub import AudioSegment
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os
import wave


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


$$
\mathbf{D}_{\text{padded}} = \mathbf{D} \parallel \left( \frac{b - (|\mathbf{D}| \mod b)}{b} \right)
$$
$$
\mathbf{IV} = \text{Random}(16)
$$
$$
\mathbf{C} = \text{AES}_{\text{CBC}}(\mathbf{K}, \mathbf{IV}, \mathbf{D}_{\text{padded}})
$$
$$
\mathbf{C}_{\text{final}} = \mathbf{IV} \parallel \mathbf{C}
$$


In [3]:
# This function reads an audio file and returns its raw data, sampling rate, sample width, and number of channels.
def read_audio_file(file_path):
    audio = AudioSegment.from_file(file_path)
    return audio.raw_data, audio.frame_rate, audio.sample_width, audio.channels

def encrypt_audio(audio_data, key):
    # Creating a random initial vector (IV) 16 bytes long
    iv = os.urandom(16)
    # Creating a cryptographic object using the AES algorithm and CBC mode.
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    # Create a padding object to add padding to the data.
    encryptor = cipher.encryptor()
    # Add padding to voice data.
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(audio_data) + padder.finalize()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return iv + encrypted_data

def save_as_wav(file_path, audio_data, frame_rate, sample_width, channels):
    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(frame_rate)
        wf.writeframes(audio_data)

def save_key(file_path, key):
    with open(file_path, 'wb') as key_file:
        key_file.write(key)


audio_data, frame_rate, sample_width, channels = read_audio_file('voice.wav')

# generate key
key = os.urandom(32)

# encrypt audio
encrypted_audio = encrypt_audio(audio_data, key)

save_as_wav('encrypted_audio.wav', encrypted_audio, frame_rate, sample_width, channels)

save_key('encryption_key.key', key)


In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import wave
from pydub import AudioSegment

def decrypt_audio(encrypted_audio, key):
    iv = encrypted_audio[:16]
    encrypted_data = encrypted_audio[16:]
    
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    decrypted_padded_data = decryptor.update(encrypted_data) + decryptor.finalize()
    
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    audio_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()
    
    return audio_data

def load_key(file_path):
    with open(file_path, 'rb') as key_file:
        return key_file.read()

def load_wav(file_path):
    with wave.open(file_path, 'rb') as wf:
        n_channels = wf.getnchannels()
        sampwidth = wf.getsampwidth()
        framerate = wf.getframerate()
        frames = wf.readframes(wf.getnframes())
    return frames, framerate, sampwidth, n_channels


encrypted_audio, frame_rate, sample_width, channels = load_wav('encrypted_audio.wav')

key = load_key('encryption_key.key')

decrypted_audio = decrypt_audio(encrypted_audio, key)

def write_audio_file(file_path, audio_data, sample_width, frame_rate, channels):
    audio = AudioSegment(
        data=audio_data,
        sample_width=sample_width,
        frame_rate=frame_rate,
        channels=channels
    )
    audio.export(file_path, format='wav')

write_audio_file('decrypted_audio.wav', decrypted_audio, sample_width, frame_rate, channels)
